# Project 01 - Option 1: Blurring a Segment in a Video
### Storm King

This Notebook contains all code for Project 01. To run the code, select Cell > Run All and observe the final output saved as 'output.mp4' in the same directory as this notebook. To view the default video passed in as an example, open 'input.mp4'. PIL, cv2, and ffmpeg are the modules necessary to run this code. 

In [ ]:
import os, sys, shutil
from PIL import Image
from PIL import ImageFilter
import cv2
import datetime
import subprocess

In [ ]:
'''
Function to validate all parameters passed in to the blurVideo function

Args:
    file_path (str): file path to any video file stored locally
    blur_box (list): list to specify location of blur in the form [start x, start y, width, height]
    start_time (str): time to begin displaying blur in the form HH:MM:SS
    end_time (str): time to stop displaying blur in the form HH:MM:SS
  
'''
def validate(file_path, blur_box, start_time, end_time):
    video_length = 0
    zero_time_dt = datetime.datetime.strptime('00:00:00', '%H:%M:%S')
    
    #Validate File Path
    try:
        video_length = int(get_length(file_path))
        duration_time_dt = datetime.datetime.strptime(str(video_length),'%S')
    except ValueError:
        sys.exit("File not Found!")
    
    #Validate Dimensions
    vid = cv2.VideoCapture(file_path)
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    
    if not isinstance(blur_box[0], int) or not isinstance(blur_box[1], int) or not isinstance(blur_box[2], int) or not isinstance(blur_box[3], int):
        sys.exit("Only Integers are allowed for dimensions")
    
    if blur_box[0] < 0 or blur_box[1] < 0:
        sys.exit("Please only use positive numbers for dimensions")
    
    if blur_box[0] + blur_box[2] > width:
        sys.exit("Starting X: {} and Width: {} exceeds video width of {}".format(blur_box[0],blur_box[2],width))
        
    if blur_box[1] + blur_box[3] > height:
        sys.exit("Starting Y: {} and Height: {} exceeds video height of {}".format(blur_box[1],blur_box[3],height))
    
    if blur_box[0] < 0 or blur_box[1] < 0:
        sys.exit("Start x and start y should be a positive")
    
    #Validate Start & End Time
    try:
        start_time_dt = datetime.datetime.strptime(start_time,'%H:%M:%S')
        end_time_dt = datetime.datetime.strptime(end_time,'%H:%M:%S')
    except ValueError:
        sys.exit("Please use format HH:MM:SS for start and end time")
    
    if start_time_dt < zero_time_dt:
        sys.exit("Start time cannot be less than 00:00:00!")
    if end_time_dt == start_time_dt:
        sys.exit("Start and End Time should NOT be the same!")
    if start_time_dt > end_time_dt:
        sys.exit("Start time cannot be greater than end time!")
    if start_time_dt > duration_time_dt:
        sys.exit("Start time is longer than the duration!")
    if end_time_dt > duration_time_dt:
        sys.exit("End time is longer than the duration!")
    
    #Ensure necessary directories exist
    try:  
        os.mkdir('./after_imgs')  
    except OSError as error:  
        print(error)
    try:  
        os.mkdir('./before_imgs')  
    except OSError as error:  
        print(error)
    try:  
        os.mkdir('./blurring_imgs_selection')  
    except OSError as error:  
        print(error)
    try:  
        os.mkdir('./final_imgs')  
    except OSError as error:  
        print(error)

In [ ]:
'''
Function to clear all files from a specified folder

Args:
    path_to_folder (str): file path to the directory to clear   
'''
def clear_folder(path_to_folder):
    folder = path_to_folder
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
'''
Function to clear all data from previous runs of blurVideo function
'''
def clear_all():
    clear_folder('./before_imgs')
    clear_folder('./after_imgs')
    clear_folder('./blurring_imgs_selection')
    clear_folder('./final_imgs')
    if os.path.exists("output.mp4"):
        os.remove("output.mp4")

In [ ]:
'''
Function to retrieve the duration of a specified video in seconds

Args:
    filename (str): file path to the video
    
Returns:
    length of the video (in seconds)
'''
def get_length(filename):
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

In [ ]:
'''
Function to segment the frames of a video based on the start and end times of the blur.
Depending on the number of segments necessary, frames from the video are placed into three
directories: before_imgs, blurring_imgs_selection, and after_imgs.

Args:
  start (str): Start time of the blur in the format HH:MM:SS
  end (str): End time of the blur in the format HH:MM:SS
  path (str): file path to the directory to clear   
'''
def split_into_segments(start, end, path):
    start_time = datetime.datetime.strptime(start,'%H:%M:%S')
    end_time = datetime.datetime.strptime(end,'%H:%M:%S')
    
    beginning_String = '00:00:00'
    video_length = int(get_length(path))
    duration_time = datetime.datetime.strptime(str(video_length),'%S')
    duration_String = duration_time.isoformat()[-8:]
    
    file='mylist.txt'
    
    #3 Segments:
    if start != beginning_String and end_time < duration_time:
        os.system("ffmpeg -ss " + beginning_String + " -to " + start + " -i " + path + " -r 30 before_imgs/image%04d.jpg")
        os.system("ffmpeg -ss " + start + " -to " + end + " -i " + path + " -r 30 blurring_imgs_selection/image%04d.jpg")
        os.system("ffmpeg -ss " + end + " -to " + duration_String + " -i " + path + " -r 30 after_imgs/image%04d.jpg")
        
        with open(file, 'w') as filetowrite:
            filetowrite.write("file './before_imgs/image%04d.jpg'\n")
            filetowrite.write("file './final_imgs/image%04d.jpg'\n")
            filetowrite.write("file './after_imgs/image%04d.jpg'\n")
    
    #2 Segments:
    elif start == beginning_String and end_time < duration_time:
        os.system("ffmpeg -ss " + start + " -to " + end + " -i " + path + " -r 30 blurring_imgs_selection/image%04d.jpg")
        os.system("ffmpeg -ss " + end + " -to " + duration_String + " -i " + path + " -r 30 after_imgs_selection/image%04d.jpg")
        
        with open(file, 'w') as filetowrite:
            filetowrite.write("file './final_imgs/image%04d.jpg'\n")
            filetowrite.write("file './after_imgs/image%04d.jpg'\n")
    
    elif start != beginning_String and end_time == duration_time:
        os.system("ffmpeg -ss " + beginning_String + " -to " + start + " -i " + path + " -r 30 before_imgs/image%04d.jpg")
        os.system("ffmpeg -ss " + start + " -to " + end + " -i " + path + " -r 30 blurring_imgs_selection/image%04d.jpg")
        
        with open(file, 'w') as filetowrite:
            filetowrite.write("file './before_imgs/image%04d.jpg'\n")
            filetowrite.write("file './final_imgs/image%04d.jpg'\n")
    
    #1 Segment:
    elif start == beginning_String and end_time == duration_time:
        os.system("ffmpeg -ss " + start + " -to " + end + " -i " + path + " -r 30 blurring_imgs_selection/image%04d.jpg")
        
        with open(file, 'w') as filetowrite:
            filetowrite.write("file './final_imgs/image%04d.jpg'\n")
    
    #ERROR
    else:
        print("ERROR: INVALID START AND END PARAMETERS")
    
    

In [ ]:
'''
Function to blur an individual frame based on the dimensions passed into blur_video.
Final results are saved to the final_imgs/ directory.

Args:
  image (PIL.Image): Image representation of the frame to blur.
  blur_dimensions (list): list to specify location of blur in the form [start x, start y, width, height]
  blur_factor (float): the starting factor used to specify the strength of GaussianBlur 
  intensity (int): value to determine the frequency of Gaussian Blurs to be performed on the image. 
'''
def blurFrame(image, blur_dimensions, blur_factor, intensity):
    #Get size of blurred area
    width = blur_dimensions[2]
    height = blur_dimensions[3]
    #Initialize Beginning/End X and Y
    begin_x_sub = blur_dimensions[0]
    begin_y_sub = blur_dimensions[1]
    end_x_sub = begin_x_sub + width
    end_y_sub = begin_y_sub + height
    
    #Gradually Perform Blurring based on Intensity & Blur Factor
    while begin_x_sub < end_x_sub and begin_y_sub < end_y_sub:
        #Blur only a portion of the image
        crop_box = [begin_x_sub, begin_y_sub, end_x_sub, end_y_sub]
        img_cropped = image.crop(crop_box)
        blurred = img_cropped.filter(ImageFilter.GaussianBlur(radius = blur_factor))
        image.paste(blurred, crop_box)
        
        #Update the blur amount
        blur_factor += 0.05
        
        #Update the subset values
        begin_x_sub = begin_x_sub + (width // intensity)
        begin_y_sub = begin_y_sub + (height // intensity)
        end_x_sub = end_x_sub - (width // intensity)
        end_y_sub = end_y_sub - (height // intensity)

    image.save('final_imgs/' + image.filename[-13:])

In [ ]:
'''
Function to blur a specific portion of a video for a specified time. 
Outputs the final blurred video as 'output.mp4' in the same directory as this file.
Default values are set for demonstration purposes, but any video can be used. 

Args:
    path (str): file path to any video file stored locally
    dimension (list): list to specify location of blur in the form [start x, start y, width, height]
    start (str): time to begin displaying blur in the form HH:MM:SS
    end (str): time to stop displaying blur in the form HH:MM:SS
  
'''
def blurVideo(path="input.mp4", dimension=[950, 300, 2500, 1700], start='00:00:03', end='00:00:23'):
    #Validate user input
    validate(path, dimension, start, end)
    
    #Make sure all folders and output is cleared from previous executions
    clear_all()
    
    #Segment the video based on blur selection
    print("Splitting video into segments...")
    split_into_segments(start, end, path)

    #Determine number of output images generated - ASSUMPTION <10000
    onlyfiles = next(os.walk('blurring_imgs_selection/'))[2]
    
    #Do Blur animation for all frames and save to final_imgs
    print("Blurring video...")
    intensity_value = 1
    increment_flag = True
    
    for file_count in range(1, len(onlyfiles)):
        #Open image depending on file count
        if file_count < 10:
            image = Image.open("blurring_imgs_selection/image000" + str(file_count) + ".jpg")
        elif file_count < 100:
            image = Image.open("blurring_imgs_selection/image00" + str(file_count) + ".jpg")
        elif file_count < 1000:
            image = Image.open("blurring_imgs_selection/image0" + str(file_count) + ".jpg")
        else:
            image = Image.open("blurring_imgs_selection/image" + str(file_count) + ".jpg")
        
        #Blur individual frame and save to final_imgs folder
        blurFrame(image, dimension, 0.5, intensity_value)
        
        #Increment/Decrement intensity value
        if increment_flag:
            intensity_value += 3
        else:
            intensity_value -= 3
        
        #Change flag if necessary
        if intensity_value >= 90:
            increment_flag = False
        elif intensity_value <= 3:
            increment_flag = True
          
        #Display Progress Indicator
        percentage_complete = (file_count / len(onlyfiles)) * 100
        sys.stdout.write('\r')
        sys.stdout.write("%d%% complete" % (percentage_complete))
        sys.stdout.flush()
    
    #Output the results of blur to output.mp4
    print("\nDone! Writing results to 'output.mp4'...")
    os.system("ffmpeg -f concat -safe 0 -i mylist.txt -r 30 output.mp4")

# Default Usage Example
Below is an example that will run blurVideo() on the default video 'input.mp4' stored as an example in this directory. 

In [ ]:
blurVideo()

# Usage Example
Below is an example that will run blurVideo() on the default video, but specifies different parameters than the default.

In [ ]:
# blurVideo("input.mp4", [900, 200, 2700, 1800], '00:00:00', '00:00:15')